In [30]:
from eval import *
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.metrics.cluster import *
from sklearn.datasets import fetch_20newsgroups
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pandas as pd
from tqdm import tqdm
import csv
torch.cuda.set_device(7)
#pd.set_option('display.max_colwidth', -1)

In [15]:
taboola_data_dir = "/local/diq/all/StepContent__nocontent_title_desc"
step_lines = "/local/diq/all/StepLines__nocontent_title_desc"

import os
print(os.listdir(step_lines))

def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

['part000004', 'part000073', 'part000109', 'part000079', 'part000103', 'part000097', 'part000041', 'part000036', 'part000110', 'part000084', 'part000017', 'part000060', 'part000058', 'part000052', 'part000025', 'part000028', 'part000022', 'part000055', 'part000083', 'part000067', 'part000010', 'part000089', 'part000031', 'part000046', 'part000074', 'part000003', 'part000090', 'part000104', 'part000009', 'part000100', 'part000094', 'part000070', 'part000007', 'part000048', 'part000035', 'part000042', 'part000063', 'part000014', 'part000113', 'part000087', 'part000069', 'part000026', 'part000051', 'part000056', 'part000021', 'part000013', 'part000064', 'part000019', 'part000080', 'part000114', 'part000038', 'part000045', 'part000032', 'part000093', 'part000107', 'part000099', 'part000077', 'part000047', 'part000030', 'part000091', 'part000105', 'part000008', 'part000002', 'part000075', 'part000054', 'part000023', 'part000029', 'part000011', 'part000066', 'part000088', 'part000082', 'part

In [27]:
for file in os.listdir(step_lines):
    sc_len = file_len("{}/{}".format(taboola_data_dir, file))
    sl_len = file_len("{}/{}".format(step_lines, file))
    print(sc_len, sl_len)
    assert sc_len == sl_len

5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
4893 4893
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000
5000 5000


KeyboardInterrupt: 

In [3]:
taxonomies = set()

#hack for the moment (jupuyter nb in folder)
for file in tqdm(os.listdir(step_lines)):
    with open("{}/{}".format(taboola_data_dir, file), "r") as f:
        data = set(map(lambda json_obj: json_obj.get('first_level_taxonomy'), map(lambda x: json.loads(x), f.readlines())))
        taxonomies= taxonomies.union(data)

print(taxonomies)
mapping = dict([(name, i) for i, name in enumerate(taxonomies)])
print(mapping)

100%|██████████| 114/114 [00:09<00:00, 11.63it/s]

{'sports', 'style and fashion', 'science', 'shopping', 'automotive and vehicles', 'business and industrial', 'food and drink', 'hobbies and interests', 'travel', 'education', 'society', 'family and parenting', 'law, govt and politics', 'real estate', 'pets', 'news', 'finance', 'health and fitness', 'home and garden', 'technology and computing', 'careers', 'art and entertainment', 'religion and spirituality'}
{'sports': 0, 'style and fashion': 1, 'science': 2, 'shopping': 3, 'automotive and vehicles': 4, 'business and industrial': 5, 'food and drink': 6, 'hobbies and interests': 7, 'travel': 8, 'education': 9, 'society': 10, 'family and parenting': 11, 'law, govt and politics': 12, 'real estate': 13, 'pets': 14, 'news': 15, 'finance': 16, 'health and fitness': 17, 'home and garden': 18, 'technology and computing': 19, 'careers': 20, 'art and entertainment': 21, 'religion and spirituality': 22}


In [34]:
text = []
labels = []

for file in tqdm(sorted(os.listdir(step_lines))):
    data = pd.read_csv("{}/{}".format(step_lines, file), header=None, sep='\t', quoting=csv.QUOTE_NONE)[2].tolist()
    text+=data
    
    with open("{}/{}".format(taboola_data_dir, file), "r") as f:
        data = map(lambda json_obj: mapping.get(json_obj.get('first_level_taxonomy'), -1), map(lambda x: json.loads(x), f.readlines()))
        labels += list(data)
        
    #print(len(text), len(labels))
    assert(len(text) == len(labels))

print(len(labels))
print(len(text))

100%|██████████| 114/114 [00:16<00:00,  7.41it/s]

569893
569893


In [35]:
test_batch_size=5000
size = len(labels)
print("Loaded dataset {} with total lines: {}".format("TABOOLA", size))

Loaded dataset TABOOLA with total lines: 569893


In [7]:
#load qt model
checkpoint_dir = '/home/jcjessecai/workspace/taboola/quickthoughts/checkpoints'
with open("{}/config.json".format(checkpoint_dir)) as fp:
    CONFIG = json.load(fp)

WV_MODEL = api.load(CONFIG['embedding'])
qt = QuickThoughts(WV_MODEL, hidden_size=CONFIG['hidden_size'])
trained_params = torch.load("{}/checkpoint_latest.pth".format(checkpoint_dir))
qt.load_state_dict(trained_params['state_dict'])
qt = qt.cuda()
qt.eval()
print("Restored successfully from {}".format(checkpoint_dir))

2019-07-18 15:00:40 INFO     loading projection weights from /home/jcjessecai/gensim-data/glove-wiki-gigaword-300/glove-wiki-gigaword-300.gz
/home/jcjessecai/workspace/miniconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2019-07-18 15:02:46 INFO     loaded (400000, 300) matrix from /home/jcjessecai/gensim-data/glove-wiki-gigaword-300/glove-wiki-gigaword-300.gz
/home/jcjessecai/workspace/miniconda3/lib/python3.7/site-packages/torch/nn/modules/rnn.py:54: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Restored successfully from /home/jcjessecai/workspace/taboola/quickthoughts/checkpoints


In [36]:
#encode data
def make_batch(j):
    """Processes one test batch of the test datset"""
    stop_idx = min(size, j+test_batch_size)
    batch_text, batch_labels  = text[j:stop_idx], labels[j:stop_idx]
    data = list(map(lambda x: torch.LongTensor(prepare_sequence(x, WV_MODEL.vocab, no_zeros=True)), batch_text))
    for i in data:
        if len(i) == 0:
            print(i)
            input()
    packed = safe_pack_sequence(data).cuda()
    return qt(packed).cpu().detach().numpy()

feature_list = [make_batch(i) for i in range(0, size, test_batch_size)]
print("Processed {:5d} batches of size {:5d}".format(len(feature_list), test_batch_size))
qt_features = np.concatenate(feature_list)
print("Test feature matrix of shape: {}".format(qt_features.shape))

Processed   114 batches of size  5000
Test feature matrix of shape: (569893, 1000)


In [42]:
taboola_model_dir = "/home/jcjessecai/workspace/taboola_model"
d2v = Doc2Vec.load("{}/{}".format(taboola_model_dir, "gensim_doc2vec.model"))

2019-07-18 16:06:20 INFO     loading Doc2Vec object from /home/jcjessecai/workspace/taboola_model/gensim_doc2vec.model
/home/jcjessecai/workspace/miniconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2019-07-18 16:06:22 INFO     loading vocabulary recursively from /home/jcjessecai/workspace/taboola_model/gensim_doc2vec.model.vocabulary.* with mmap=None
2019-07-18 16:06:22 INFO     loading trainables recursively from /home/jcjessecai/workspace/taboola_model/gensim_doc2vec.model.trainables.* with mmap=None
2019-07-18 16:06:22 INFO     loading syn1neg from /home/jcjessecai/workspace/taboola_model/gensim_doc2vec.model.trainables.syn1neg.npy with mmap=None
2019-07-18 16:06:25 INFO     loading wv 

In [43]:
d2v_features = np.vstack([d2v.infer_vector(doc) for doc in text])
print(d2v_features.shape)

(569893, 600)


In [ ]:
#first we compare embedding performance by fitting binary classifier on top

from sklearn import linear_model

X_train, X_test, y_train, y_test = train_test_split(d2v_features, labels)
clf = linear_model.SGDClassifier(loss='log', max_iter=1000, tol=1e-3, n_jobs=20)
clf.fit(X_train, y_train)
train_acc = clf.score(X_train, y_train)
test_acc = clf.score(X_test, y_test)
print("Fit logistic model on d2v with s: {:3d} and train acc: {:.2%} test acc: {:.2%}".format(s, train_acc, test_acc))


X_train, X_test, y_train, y_test = train_test_split(qt_features, labels)
clf = linear_model.SGDClassifier(loss='log', max_iter=1000, tol=1e-3, n_jobs=20)
clf.fit(X_train, y_train)
train_acc = clf.score(X_train, y_train)
test_acc = clf.score(X_test, y_test)
print("Fit logistic model on qt with s: {:3d} and train acc: {:.2%} test acc: {:.2%}".format(s, train_acc, test_acc))


In [ ]:
qt_predicted  = KMeans(n_clusters=7, n_jobs=20).fit_predict(qt_features)
d2v_predicted  = KMeans(n_clusters=7, n_jobs=20).fit_predict(d2v_features)

In [ ]:
print(adjusted_rand_score(top_level_labels, qt_predicted))
print(adjusted_rand_score(top_level_labels, d2v_predicted))

print(adjusted_mutual_info_score(top_level_labels, qt_predicted))
print(adjusted_mutual_info_score(top_level_labels, d2v_predicted))

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

feature_embedded = TSNE(n_components=2, verbose=1).fit_transform(features[:5000])

fig, axs = plt.subplots(4, 5, figsize=(20, 15))

fixed_xlim, fixed_ylim = (-85.6129455010451, 110.84618372125996), (-90.00594782812799, 80.691349744632)

for i in range(20):
    selected = feature_embedded[labels[:5000] == i]
    ax = axs[i//5, i%5]
    ax.scatter(selected[:, 0], selected[:, 1])
    ax.set_ylim(fixed_ylim)
    ax.set_xlim(fixed_xlim)
    
plt.show()

fig, axs = plt.subplots(4, 5, figsize=(20, 15))

fixed_xlim, fixed_ylim = (-85.6129455010451, 110.84618372125996), (-90.00594782812799, 80.691349744632)

for i in range(20):
    selected = feature_embedded[predicted_spectral[:5000] == i]
    ax = axs[i//5, i%5]
    ax.scatter(selected[:, 0], selected[:, 1])
    ax.set_ylim(fixed_ylim)
    ax.set_xlim(fixed_xlim)
    
plt.show()

fig, axs = plt.subplots(4, 5, figsize=(20, 15))

fixed_xlim, fixed_ylim = (-85.6129455010451, 110.84618372125996), (-90.00594782812799, 80.691349744632)

for i in range(20):
    selected = feature_embedded[predicted_kmeans[:5000] == i]
    ax = axs[i//5, i%5]
    ax.scatter(selected[:, 0], selected[:, 1])
    ax.set_ylim(fixed_ylim)
    ax.set_xlim(fixed_xlim)
    
plt.show()